In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 29.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=798b51901c504d01b7aedb31ad0f1af30eaa4dc4637bbe0616b7e388957656da
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import tensorflow

In [4]:
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [5]:
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import numpy as np 
import random 
import json 

In [7]:
with open("intents.json") as file :
  data = json.load(file)

In [8]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'contact', 'patterns': ['can you send me your concat details ?', 'phone number', 'how can i contact ?', 'how cani call you ?', 'phone?', 'emeil?', 'location ?', 'where are you ?', 'where can i find you ?', 'adress ?', 'mail ?', 'help', 'i need help', 'can you give me support ?'], 'responses': ['You can contact us via Email :\nLocation:\n\t 401, Shreenath Complex\nVadgaon Pune India-41\n Email:\n\tinfo@codeclause.com\n official@codeclause.com \nCall:\n\t+91 7030020973', 'you can contact us on : https://codeclause.com/Contact'], 'context_set': ''}, {'tag': 'identity', 'patterns': ['w

In [9]:
docs_x = []
docs_y = []

In [10]:
words = []
labels = []
docs = []

for intent in data['intents']:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds) #words.append(wrds)
    docs_x.append(wrds)
    docs_y.append(intent['tag'])

    if intent['tag'] not in labels :
      labels.append(intent['tag'])


In [11]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

In [12]:
labels = sorted(labels)

In [13]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
  bag= []

  wrds = [stemmer.stem(w) for w in doc]

  for w in words :
    if w in wrds :
      bag.append(1)
    else :
      bag.append(0)
  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] =1

  training.append(bag)
  output.append(output_row)


In [14]:
training = np.array(training)
output = np.array(output)

In [15]:
net =tflearn.input_data(shape =[None ,len(training[0])])
net =tflearn.fully_connected(net ,8)
net =tflearn.fully_connected(net ,8)
net =tflearn.fully_connected(net ,len(output[0]), activation ="softmax")
tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'FullyConnected_2/Softmax:0' shape=(?, 6) dtype=float32>

In [16]:
model = tflearn.DNN(net)

In [17]:
model.fit(training , output, n_epoch = 1000,batch_size = 8 ,show_metric =True)

Training Step: 5999  | total loss: 0.00640 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00640 - acc: 1.0000 -- iter: 40/48
Training Step: 6000  | total loss: 0.00618 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.00618 - acc: 1.0000 -- iter: 48/48
--


In [18]:
model.save("model.tflearn")

In [19]:
from nltk import tokenize

In [23]:
def bag_of_words(s ,words) :
  bag = [0 for _ in range(len(words))]

  s_words = tokenize.word_tokenize(s)

  s_words = [stemmer.stem(word.lower()) for word in s_words]


  for se in s_words :
    for i, w in enumerate(words) :
      if w == se :
        # print(bag)
        bag[i] = 1
  # print(np.array(bag).shape)
  return np.array(bag).reshape((1,67))

In [24]:
def chat() :
  print('start chat with bot')
  while True :
    inp = input('You : ')
    if inp.lower() == "quit" :
      break
    # print(inp)
    results = model.predict(bag_of_words(inp,words))
    # print(results)
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data['intents'] :
      if tg['tag'] == tag :
        responses = tg['responses']
    print(random.choice(responses))

In [25]:
chat()

start chat with bot
You : hello
Hello!
You : how are you ?
Good to see you again!
You : who are you ?
Hey!! 
We are a trusted Software Development and IT Services, combining deep technical expertise and industry experience to help our clients anticipate what’s next and answer questions before they’re asked. Our offerings and proven solutions create unique competitive advantage for our clients by giving them the power to see beyond and rise above
You : services?
We provide Data & Analytics 
Automation AI/ML 
Internet of Things(IoT) 
Cloud Solutions
You : do you have something else?
We have 4 products : 
PDFSuite 
Transfy 
Attendit 
DigiCracker 
You : thanks
Hey!! 
We are a trusted Software Development and IT Services, combining deep technical expertise and industry experience to help our clients anticipate what’s next and answer questions before they’re asked. Our offerings and proven solutions create unique competitive advantage for our clients by giving them the power to see beyond an